In [ ]:
!pip install praw nltk
# get credentioals from here https://www.reddit.com/prefs/apps/

import logging

# Set logging level to suppress PRAW warning messages
logging.getLogger("praw").setLevel(logging.WARNING)
logging.getLogger("praw").setLevel(logging.ERROR)

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
import praw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Reddit API credentials
reddit_client_id = 'bLfHRgL3R8y69_QUj7xMrA'
reddit_client_secret = 'HO0fvx3tIbnujuyCqIPmjXT9lopiCg'
reddit_user_agent = 'CommunicationTop2560'

# Initialize Reddit API
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Subreddit to follow
subreddit_name = 'gadgets'
subreddit = reddit.subreddit(subreddit_name)

# Extracting data from the subreddit
posts = []
for submission in subreddit.top('year', limit=10000):  # Example: Top posts of the month, limit to 1000
    posts.append({
        'ID': submission.id,
        'Title': submission.title,
        'Score': submission.score,
        'URL': submission.url,
        'Num_comments': submission.num_comments,
        'Created_utc': submission.created_utc,
        'Author':submission.author,
    })

# Create a DataFrame from the extracted data
df = pd.DataFrame(posts)

# Save the DataFrame to a CSV file
# df.to_csv('gadgets_data.csv', index=False)

# Display the DataFrame
df

,ID,Title,Score,URL,Num_comments,Created_utc,Author
0,10upla2,"Couple used Apple AirTag to track luggage, fou...",51235,https://www.pcmag.com/news/couple-used-apple-a...,1818,1.675636e+09,thebelsnickle1991
1,12x30gv,Scalpers are struggling to sell PlayStation 5 ...,47861,https://www.techspot.com/news/98403-scalpers-s...,2636,1.682310e+09,thebelsnickle1991
2,14ddlcs,EU: Smartphones Must Have User-Replaceable Bat...,36832,https://www.pcmag.com/news/eu-smartphones-must...,3161,1.687178e+09,FigAAAro_22
3,10w0qp4,The Switch has now outsold the PS4 and Game Bo...,31390,https://www.engadget.com/the-switch-has-outsol...,1813,1.675773e+09,chrisdh79
4,11c8ojr,Nokia is supporting a user's right-to-repair b...,29540,https://www.digitaltrends.com/mobile/hmd-globa...,1024,1.677398e+09,MicroSofty88
...,...,...,...,...,...,...,...
995,12hecfr,Pixel 7a leaks in official-looking images show...,145,https://9to5google.com/2023/04/09/pixel-7a-ren...,47,1.681122e+09,No-Drawing-6975
996,191n2zx,Samsung's 990 EVO SSD could support both PCIe ...,142,https://www.techspot.com/news/101439-samsung-9...,11,1.704726e+09,chrisdh79
997,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,146,https://www.macrumors.com/2023/02/28/iphone-16...,42,1.677591e+09,chrisdh79
998,184nrlt,AMD Radeon RX 7900M performance debut: Alienwa...,142,https://www.notebookcheck.net/AMD-Radeon-RX-79...,20,1.701040e+09,Stiven_Crysis


In [ ]:
# Extracting comments from posts with titles containing 'iphone'
filtered_posts = df[df['Title'].str.contains('iphone', case=False, regex=True)]
selected_post_ids = filtered_posts['ID'].tolist()

iphone_comments_data = []

for post_id in selected_post_ids:
    try:
        post = reddit.submission(id=post_id)
        post_title = df[df['ID'] == post_id]['Title'].values[0]
        comments = post.comments.list()

        # Extracting more information from comments
        for comment in comments:
            if isinstance(comment, praw.models.MoreComments):
                continue  # Skip 'MoreComments' objects
            iphone_comments_data.append({
                'Post_ID': post_id,
                'Title': post_title,
                'Comment': comment.body,
                'Created_utc': comment.created_utc,
                'Author': str(comment.author) if comment.author else '[deleted]',
                'Upvote_ratio': post.upvote_ratio,
                'Score': post.score,
                'Subreddit': post.subreddit.display_name,
                'URL': post.url,
                'Domain': post.domain,
                'Media': post.media,
                'Is_video': post.is_video,
                'Thumbnail': post.thumbnail,
                'Permalink': post.permalink,
                'Selftext': post.selftext,
                'Upvotes': post.ups,
            })
    except Exception as e:
        print(f"Error processing post {post_id}: {str(e)}")

# Create a DataFrame from the extracted iPhone comments data
iphone_comments_df = pd.DataFrame(iphone_comments_data)
iphone_comments_df

,Post_ID,Title,Comment,Created_utc,Author,Upvote_ratio,Score,Subreddit,URL,Domain,Media,Is_video,Thumbnail,Permalink,Selftext,Upvotes
0,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",They’d have to allow data transfer over MagSaf...,1.673752e+09,Grantsdale,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608
1,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",It’s like a Barbie doll.,1.673749e+09,victim_of_technology,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608
2,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",why is it titled like they've already made a p...,1.673749e+09,deweydean,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608
3,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",This entire article is just someone making shi...,1.673753e+09,[deleted],0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608
4,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",Here's how this works\n\n1. Apple presents ide...,1.673751e+09,madogson,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31112,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,I loved the button/sensor on the back of by g6...,1.677708e+09,Datkif,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146
31113,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,"With deepfakes, I would worry about voice ID\n...",1.677609e+09,What-a-Crock,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146
31114,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Doesn’t FaceId have a mask mode now?,1.677612e+09,SmurphsLaw,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146
31115,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Yeah I had it on my s8. Super fast and the pla...,1.677711e+09,snozzcumbersoup,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146


In [ ]:
# Combine Title and Selftext for text analysis
iphone_comments_df['Text'] = iphone_comments_df['Title'] + ' ' + iphone_comments_df['Comment'] + ' ' + iphone_comments_df['Selftext']

# Tokenize and clean the text
stop_words = set(stopwords.words('english'))
iphone_comments_df['Tokens'] = iphone_comments_df['Text'].apply(lambda x: word_tokenize(re.sub(r'[^a-zA-Z0-9]', ' ', str(x).lower())))
iphone_comments_df['Tokens'] = iphone_comments_df['Tokens'].apply(lambda x: [word for word in x if word.isalnum() and word not in stop_words])

# Flatten the list of tokens
all_tokens = [token for sublist in iphone_comments_df['Tokens'].tolist() for token in sublist]
token_series = pd.Series(all_tokens)
token_counts = token_series.value_counts()

In [ ]:
# Identify keywords and hashtags using frequency distribution
fdist = FreqDist(all_tokens)
keywords = [word for word, freq in fdist.most_common(20) if len(word) > 2]

# Identify influencers (top authors)
top_authors = iphone_comments_df['Author'].value_counts().index[:5].tolist()

# Identify different groups (subreddits)
different_groups = iphone_comments_df['Subreddit'].unique().tolist()

# Display the results
print("Keywords and Hashtags:", keywords)
print("\nTop Influencers:", top_authors)
print("\nDifferent Groups (Subreddits):", different_groups)

Keywords and Hashtags: ['iphone', 'apple', 'pro', 'usb', 'new', 'phone', 'iphones', 'like', 'people', 'charging', 'max', 'port', 'battery', 'year', 'get', 'android', 'use', 'would']

Top Influencers: ['[deleted]', 'nicuramar', 'SUPRVLLAN', 'Sylvurphlame', 'Pubelication']

Different Groups (Subreddits): ['gadgets']


In [ ]:
token_counts

iphone         32942
apple          23601
15             14694
pro            10590
usb             9827
               ...  
bwm                1
quantify           1
attribution        1
behavioral         1
deepfake           1
Length: 24306, dtype: int64

In [ ]:
# Identify keywords and hashtags using frequency distribution
fdist = FreqDist(all_tokens)
keywords = [word for word, freq in fdist.most_common(20) if len(word) > 2]

# Identify influencers (top authors)
top_authors = iphone_comments_df['Author'].value_counts().index[:5].tolist()

# Identify different groups (subreddits)
different_groups = iphone_comments_df['Subreddit'].unique().tolist()

In [ ]:
# Display the results
print("Keywords and Hashtags:", keywords)
print("\nTop Influencers:", top_authors)
print("\nDifferent Groups (Subreddits):", different_groups)

Keywords and Hashtags: ['iphone', 'apple', 'pro', 'usb', 'new', 'phone', 'iphones', 'like', 'people', 'charging', 'max', 'port', 'battery', 'year', 'get', 'android', 'use', 'would']

Top Influencers: ['[deleted]', 'nicuramar', 'SUPRVLLAN', 'Sylvurphlame', 'Pubelication']

Different Groups (Subreddits): ['gadgets']


In [ ]:
iphone_comments_df

,Post_ID,Title,Comment,Created_utc,Author,Upvote_ratio,Score,Subreddit,URL,Domain,Media,Is_video,Thumbnail,Permalink,Selftext,Upvotes,Text,Tokens
0,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",They’d have to allow data transfer over MagSaf...,1.673752e+09,Grantsdale,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
1,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",It’s like a Barbie doll.,1.673749e+09,victim_of_technology,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
2,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",why is it titled like they've already made a p...,1.673749e+09,deweydean,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
3,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",This entire article is just someone making shi...,1.673753e+09,[deleted],0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
4,10c7dvd,"Sorry, Apple — a portless iPhone is a terrible...",Here's how this works\n\n1. Apple presents ide...,1.673751e+09,madogson,0.83,24608,gadgets,https://www.digitaltrends.com/mobile/apple-iph...,digitaltrends.com,None,False,https://b.thumbs.redditmedia.com/JA00tdSa_G4MV...,/r/gadgets/comments/10c7dvd/sorry_apple_a_port...,,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31112,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,I loved the button/sensor on the back of by g6...,1.677708e+09,Datkif,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31113,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,"With deepfakes, I would worry about voice ID\n...",1.677609e+09,What-a-Crock,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31114,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Doesn’t FaceId have a mask mode now?,1.677612e+09,SmurphsLaw,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD765q6ub8...,/r/gadgets/comments/11e67uz/iphone_16_pro_stil...,,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31115,11e67uz,iPhone 16 Pro Still on Track to Feature Under-...,Yeah I had it on my s8. Super fast and the pla...,1.677711e+09,snozzcumbersoup,0.80,146,gadgets,https://www.macrumors.com/2023/02/28/iphone-16...,macrumors.com,None,False,https://b.thumbs.redditmedia.com/jkPeD7

In [ ]:
columns_to_drop = ['Post_ID', 'Created_utc', 'Subreddit', 'URL', 'Media', 'Domain', 'Is_video', 'Thumbnail', 'Permalink', 'Selftext']

# Drop the specified columns
df = iphone_comments_df.drop(columns=columns_to_drop)
df

,Title,Comment,Created_utc,Author,Upvote_ratio,Score,Upvotes,Text,Tokens
0,"Sorry, Apple — a portless iPhone is a terrible...",They’d have to allow data transfer over MagSaf...,1.673752e+09,Grantsdale,0.83,24608,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
1,"Sorry, Apple — a portless iPhone is a terrible...",It’s like a Barbie doll.,1.673749e+09,victim_of_technology,0.83,24608,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
2,"Sorry, Apple — a portless iPhone is a terrible...",why is it titled like they've already made a p...,1.673749e+09,deweydean,0.83,24608,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
3,"Sorry, Apple — a portless iPhone is a terrible...",This entire article is just someone making shi...,1.673753e+09,[deleted],0.83,24608,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
4,"Sorry, Apple — a portless iPhone is a terrible...",Here's how this works\n\n1. Apple presents ide...,1.673751e+09,madogson,0.83,24608,24608,"Sorry, Apple — a portless iPhone is a terrible...","[sorry, apple, portless, iphone, terrible, ide..."
...,...,...,...,...,...,...,...,...,...
31112,iPhone 16 Pro Still on Track to Feature Under-...,I loved the button/sensor on the back of by g6...,1.677708e+09,Datkif,0.80,146,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31113,iPhone 16 Pro Still on Track to Feature Under-...,"With deepfakes, I would worry about voice ID\n...",1.677609e+09,What-a-Crock,0.80,146,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31114,iPhone 16 Pro Still on Track to Feature Under-...,Doesn’t FaceId have a mask mode now?,1.677612e+09,SmurphsLaw,0.80,146,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."
31115,iPhone 16 Pro Still on Track to Feature Under-...,Yeah I had it on my s8. Super fast and the pla...,1.677711e+09,snozzcumbersoup,0.80,146,146,iPhone 16 Pro Still on Track to Feature Under-...,"[iphone, 16, pro, still, track, feature, displ..."


In [ ]:
df = df[df['Author'] != '[deleted]']
unique_title_count = df['Title'].nunique()
unique_title_count

104

In [ ]:
title_comment_counts = df.groupby('Title')['Comment'].count()
title_comment_counts.sort_values(ascending=False)

Title
Apple isn't happy about India's demand to upgrade older iPhones with USB-C                                                                477
Apple announces that RCS support is coming to iPhone next year                                                                            472
Apple announces iPhone 15 Pro and Pro Max                                                                                                 468
No, you really don't need the iPhone 15                                                                                                   467
New iPhone, new charger: Apple bends to EU rules                                                                                          465
                                                                                                                                         ... 
Apple uses hidden watermarks on iPhone 15 boxes to verify authenticity                                                                     33


In [ ]:
title_comment_counts = iphone_comments_df.groupby('Title')['Comment'].count()

# Filter titles with a count between 100 and 200 comments
filtered_title_comment_counts = title_comment_counts[(title_comment_counts >= 100) & (title_comment_counts <= 200)]

# Print the filtered result
print(filtered_title_comment_counts)

Title
Apple Fixes Bug Allowing Flipper Zero to Lock Up iPhones                                                                                                                    123
Apple Starts Sending 'Batterygate' Settlement Payments to iPhone Users                                                                                                      171
Apple addresses iPhone 15 overheating with a new iOS 17 update.                                                                                                             138
Apple iPhone 15 Pro Max battery tests indicate worse endurance than the iPhone 14 Pro Max                                                                                   123
Apple submits update that will weaken iPhone 12 modem to French government                                                                                                  153
Apple to Issue iPhone 12 Software Update in France to Address Radiation Concerns                                  